## Compare two different genotypes

This script allows you to create a JSON file with point cloud data of two groups of animals with different genotypes (wildtype C57BL/6J and transgenic Nex-cKO-Pos) ([DOI: 10.25493/AJVE-B8G](https://search.kg.ebrains.eu/instances/27dd2588-f4da-406c-9d01-8569bf912b0e)), which can be visualized in [Meshview](https://meshview.apps.hbp.eu/?atlas=ABA_Mouse_CCFv3_2017_25um).

The WT control is depicted in blue and the Nex-cKO-Pos animal is dipicted in red.
| WT-controls | Color WT | Nex-cKO| Color Nex-cKO |
|---|---|---|---|
|112229814_SSpul | Blue [0, 0, 255] |19423_7_SSpul   | Red [255, 0, 0] |




In [1]:
# Import packages
import json
import os
from pathlib import Path
import itertools


## Define the cases of interest

In [2]:
# First select the cases that we want to compare
cases = [
    {
    "group": "WT-controls", 
    "case" : [
        {"name": "112229814_SSpul", "color": [0, 0, 255]}
        ]
    },
    {
    "group": "Nex-cKO", 
    "case" : [
        {"name": "19423_7_SSpul", "color": [255, 0, 0]}
        ]
    }
]


## Create output folder and find the experiment files
Ensure that the experiment files downloaded from EBRAINS are in the data folder

In [3]:
# Data folder
data_folder = os.path.join("..", "data")

# Check if output folder exist, if not create the folder
output_folder = os.path.join("..", "output")
if not os.path.exists(output_folder):
   os.makedirs(output_folder)
   print("The new directory is created!")

fnames = []
for group in cases:
  group_list = []
  for case in group['case']:
    expName = case['name']
    fname = sorted(Path(data_folder).rglob(f"*{expName}*.json"))
    group_list.append(fname[0])
  fnames.append(group_list)


## Create the JSON file for the comparison of point clouds based on genotype

In [4]:
points_list = []

for case in cases:
    group_name = case['group']
    for count, case_info in enumerate(case['case']):
        fname = fnames[cases.index(case)][count]
        expName = os.path.splitext(os.path.basename(fname))[0]
        
        with open(fname, 'r') as f:
            points = json.load(f)

        # select the correct area (WT ones have 4 brain regions)
        point_cloud = points[0] if group_name == "Nex-cKO" else points[2]    
        
        point_cloud.update({
            "name": expName,
            "r": int(case_info['color'][0]),
            "g": int(case_info['color'][1]),
            "b": int(case_info['color'][2])
        })
        
        points_list.append(point_cloud)

# To speed up saving of the JSON file - works well for large point clouds
def chunked_json_dump(data, file, chunk_size=1024):
    for i in range(0, len(data), chunk_size):
        file.write(json.dumps(data[i:i+chunk_size], ensure_ascii=False))

# Saving the JSON file
out_file = os.path.join(output_folder, f"compare_genotype.json")
print(f"Saving file {out_file}")
with open(out_file, 'w', encoding='utf-8') as fi:
    chunked_json_dump(points_list, fi)

print("Done!")

Saving file ..\output\compare_genotype.json
Done!
